---
toc: true
comments: true
layout: post
title: Predictive Stock Analysis
courses: { csa: {week: 2} }
type: hacks
permalink: /stocks
authors: Aditya, Parav, Rohin
---

<html>
<body>
<div>
  <!-- Input for the stock symbol -->
  <label for="stockInput">Enter Stock Symbol: </label>
  <input type="text" id="stockInput" placeholder="e.g. MSFT">
</div>

<div id="monthButtons">
  <!-- Buttons for each month will be dynamically generated here -->
</div>

<div class="dropdown" id="typeDropdown">
  <button class="dropbtn">Type</button>
  <div class="dropdown-content">
    <!-- Dropdown buttons will be dynamically generated here -->
  </div>
</div>

  <!-- Canvas element for the graph -->
  <canvas></canvas>

  <div class="prediction-wrapper" style="display:none">
    <!-- Input for user to enter a date for prediction -->
    <label for="predictionInput">Enter a Month to Predict your Selected Stock Price (YYYY-MM): </label>
    <input type="text" id="predictionInput" placeholder="e.g. 2023-09">
    <button onclick="predictPrice()">Predict Price</button>
    <!-- Display the prediction result -->
    <h1 id="prediction"></h1>
  </div>

<script>
  // Function to generate buttons for each month
  function generateMonthButtons() {
    const months = [
      { name: 'July', value: 6 },
      { name: 'August', value: 5 },
      { name: 'September', value: 4 },
      { name: 'October', value: 3 },
      { name: 'November', value: 2 },
      { name: 'December', value: 1 }
    ];

    const monthButtonsContainer = document.getElementById('monthButtons');

    months.forEach(month => {
      const button = document.createElement('button');
      button.textContent = month.name;
      button.onclick = () => fetchMonthlyData(month.value);
      monthButtonsContainer.appendChild(button);
    });
  }

  // Function to generate buttons for the dropdown
  function generateDropdownButtons() {
    const types = ['chronological', 'low-to-high', 'high-to-low'];

    const dropdownContainer = document.querySelector('#typeDropdown .dropdown-content');

    types.forEach(type => {
      const button = document.createElement('a');
      button.textContent = type.replace('-', ' ');
      button.onclick = () => fetchData(type);
      dropdownContainer.appendChild(button);
    });
  }

  // Call the functions to generate buttons
  generateMonthButtons();
  generateDropdownButtons();
    // Global variables to store regression data and x-value
    let regression = {}
    let x = 0;
    let ctx;

    // Function to fetch stock data for a specific month
    async function fetchMonthlyData(month) {
      const url = `http://localhost:8098/api/DailyStocks/${month}`;

      try {
        // Fetch stock data from the backend
        const response = await fetch(url);
        const monthlyData = await response.json();
        console.log(monthlyData);

        // Extract dates and adjusted closing prices from the API response
        const dates = monthlyData.map(entry => entry.date);
        const prices = monthlyData.map(entry => parseFloat(entry.high));

        // Generate the graph using the data
        generateGraph(dates, prices);
      } catch (error) {
        console.error(error);
      }
    }

    // Function to fetch and display all 6 months based on sorting order
    async function fetchData(orderType) {
      const stockSymbol = document.getElementById('stockInput').value;
      const url = `http://localhost:8098/api/monthly-stocks/${orderType}`;

      try {
        // Fetch stock data from the backend
        const response = await fetch(url);
        const result = await response.json();
        console.log(result);
        console.log(result.monthlyStocks);

        // Check if the result is an array
        if (Array.isArray(result)) {
          console.log("Array");
          // Iterate over each month in the result and generate a graph
          for (const monthlyData of result.monthlyStocks) {
            console.log("Array");
            const dates = monthlyData.monthlyStocks.map(entry => entry.date);
            console.log(entry.date);
            const prices = monthlyData.data.map(entry => parseFloat(entry.high));
            console.log(prices);

            // Generate the graph using the data
            generateGraph(dates, prices);
          }
        } else if (Array.isArray(result.data)) {
          console.log("test");
          // Handle the case when the result is not an array (for chronological, ascending, and descending)
          const dates = result.monthlyStocks.map(entry => entry.date);
          const prices = result.monthlyStocks.map(entry => parseFloat(entry.high));
          console.log(prices);

          // Generate the graph using the data
          generateGraph(dates, prices);
        } else {
          console.log("hello");
        }
      } catch (error) {
        console.error(error);
      }
    }

    // Function to generate the graph
    function generateGraph(dates, prices) {
      // Get canvas element and context
      const canvas = document.getElementsByTagName('canvas')[0];
      ctx = canvas.getContext('2d');

      // Define canvas dimensions
      canvas.width = 700;
      canvas.height = 400;

      // Define data points and chart dimensions
      const dataPoints = prices.length;
      const margin = 40;
      const chartWidth = canvas.width - 2 * margin;
      const chartHeight = canvas.height - 2 * margin;

      // Calculate scaling factors
      const maxPrice = Math.max(...prices);
      const minPrice = Math.min(...prices);
      const priceRange = maxPrice - minPrice;
      const xStep = chartWidth / (dataPoints - 1);
      const yStep = chartHeight / priceRange;

      // Draw x and y axes
      ctx.beginPath();
      ctx.moveTo(margin, margin);
      ctx.lineTo(margin, canvas.height - margin);
      ctx.lineTo(canvas.width - margin, canvas.height - margin);
      ctx.stroke();

      // Plot stock prices
      ctx.beginPath();
      ctx.strokeStyle = 'blue';
      ctx.lineWidth = 2;
      for (let i = dataPoints; i > 0; i--) {
        const x = canvas.width - 40 - i * xStep;
        const y = canvas.height - margin - (prices[i] - minPrice) * yStep;
        if (i === 0) {
          ctx.moveTo(x, y);
        } else {
          ctx.lineTo(x, y);
        }
      }
      ctx.stroke();

      // Calculate linear regression
      const xValues = [...Array(dataPoints).keys()];
      const yValues = prices;
      regression = linearRegression(xValues, yValues);

      // Plot linear regression line
      ctx.beginPath();
      ctx.strokeStyle = 'red';
      ctx.lineWidth = 2;
      for (let i = 0; i < dataPoints; i++) {
        const x = canvas.width - 40 - i * xStep;
        const y = canvas.height - margin - (regression.slope * i + regression.intercept - minPrice) * yStep;
        if (i === 0) {
          ctx.moveTo(x, y);
        } else {
          ctx.lineTo(x, y);
        }
      }
      ctx.stroke();

      // Label axes
      ctx.fillStyle = 'black';
      ctx.fillText('Date', canvas.width - margin - 20, canvas.height - 10);
      ctx.fillText('Stock Price', margin - 10, margin - 10);

      // Display the prediction input section
      document.getElementsByClassName("prediction-wrapper")[0].style.display = "block"

      // Calculate Fibonacci retracement levels
      const fibonacciLevels = [0, 0.236, 0.382, 0.5, 0.618, 0.786, 1];

      // Draw Fibonacci retracement lines
      ctx.beginPath();
      ctx.setLineDash([5, 5]); // Set a dashed line style

      for (const level of fibonacciLevels) {
        const yValue = maxPrice - level * priceRange;
        const y = canvas.height - margin - (yValue - minPrice) * yStep;

        ctx.moveTo(margin, y);
        ctx.lineTo(canvas.width - margin, y);
      
        // Label the line
        ctx.fillStyle = 'black';
        ctx.fillText(`${(level * 100).toFixed(1)}%`, margin - 40, y - 5); // Adjust the x-position
      }

      ctx.stroke();
      ctx.setLineDash([]); // Reset the line style
    }

    // Function to calculate linear regression
    function linearRegression(x, y) {
      const n = x.length;
      let sumX = 0;
      let sumY = 0;
      let sumXY = 0;
      let sumX2 = 0;

      for (let i = 0; i < n; i++) {
        sumX += x[i];
        sumY += y[i];
        sumXY += x[i] * y[i];
        sumX2 += x[i] * x[i];
      }

      const slope = (n * sumXY - sumX * sumY) / (n * sumX2 - sumX * sumX);
      const intercept = (sumY - slope * sumX) / n;

      return {
        slope,
        intercept
      };
    }

    // Function to calculate the index based on user input
    function calculateIndex(userInput) {
      // Split the user input into year and month
      const [year, month] = userInput.split('-');
    
      // Calculate the index based on the year and month, assuming January 2000 as the starting point
      const startIndex = (parseInt(year) - 2000) * 12 + (parseInt(month) + 1);
    
      return startIndex;
    }    

    // Function to predict the stock price
    function predictPrice() {
      const prediction = document.getElementById("predictionInput").value;

      // Calculate the x-value based on user input
      let xValue = calculateIndex(prediction)

      // Predict the stock price using linear regression formula
      const predictedPrice = (-regression.slope * xValue + regression.intercept);

      // Display the predicted price
      document.getElementById("prediction").textContent = "$" + predictedPrice.toFixed(2);
    }
  </script>

<style>

#stockInput {
  padding: 10px;
  font-size: 16px;
  border: 1px solid #ccc;
  border-radius: 5px;
  margin-right: 10px;
}

button {
  background-color: #4CAF50; /* Green background color */
  border: none;
  color: white; /* White text color */
  padding: 8px 11px; /* Padding inside the button */
  text-align: center;
  text-decoration: none;
  display: inline-block;
  font-size: 13px;
  margin: 4px 1px; /* Margin between buttons */
  cursor: pointer;
  transition-duration: 0.4s; /* Add a smooth transition effect */
  border-radius: 4px; /* Rounded corners */
}

button:hover {
  background-color: white; /* White background on hover */
  color: black; /* Black text on hover */
}

/* Style The Dropdown Button */
.dropbtn {
  background-color: black;
  color: white;
  padding: 16px;
  font-size: 16px;
  border: none;
  cursor: pointer;
}

div.dropdown{
  width: 50px;
  height: 100px;
}

/* The container <div> - needed to position the dropdown content */
.dropdown {
  position: relative;
}

/* Dropdown Content (Hidden by Default) */
.dropdown-content {
  display: none;
  position: absolute;
  background-color: #f9f9f9;
  min-width: 160px;
  box-shadow: 0px 8px 16px 0px rgba(0,0,0,0.2);
  z-index: 1;
}

/* Links inside the dropdown */
.dropdown-content a {
  color: black;
  padding: 12px 16px;
  text-decoration: none;
  display: block;
}

/* Change color of dropdown links on hover */
.dropdown-content a:hover {background-color: #f1f1f1}

/* Show the dropdown menu on hover */
.dropdown:hover .dropdown-content {
  display: block;
}

/* Change the background color of the dropdown button when the dropdown content is shown */
.dropdown:hover .dropbtn {
  background-color: #58595b;
}

/* Body Styling */
body {
  font-family: 'Arial', sans-serif;
  margin: 0;
  padding: 0;
  background-color: white; /* Dark background color */
  color:black; /* Text color */
}

/* Page Container Styling */
body > div {
  max-width: 800px;
  margin: 20px auto;
  background-color: white; /* Container background color */
  padding: 20px;
  border-radius: 10px;
  box-shadow: 0 0 10px rgba(0, 0, 0, 0.3);
}

/* Header Styling */
h1 {
  color: #4CAF50; /* Green header color */
}

</style>

